In [ ]:
# Databricks notebook source
# =============================================================================
# 🎯 FIXED MODEL EVALUATION & AUTO-PROMOTION SYSTEM
# =============================================================================
# Hard-coded configuration to match training_script.py and Model_Registration.ipynb
# =============================================================================

%pip install xgboost requests
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime
from pyspark.sql import SparkSession
import traceback

print("=" * 80)
print("🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM")
print("=" * 80)

# =============================================================================
# ✅ HARD-CODED CONFIGURATION (MUST MATCH OTHER SCRIPTS!)
# =============================================================================
# These values are DIRECTLY from training_script.py and Model_Registration.ipynb

EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"  # ✅ From training script
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"  # ✅ From registration script

STAGING_ALIAS = "Staging"
PRODUCTION_ALIAS = "production"

MODEL_ARTIFACT_PATH = "xgboost_model"  # ✅ From training script
METRIC_KEY = "test_rmse"              # ✅ From training script
IMPROVEMENT_THRESHOLD = 0.02          # 2% improvement needed for promotion

# Logging Config
COMPARISON_LOG_TABLE = "workspace.default.model_evaluation_log"

print("\n📋 CONFIGURATION:")
print(f"   Experiment Name: {EXPERIMENT_NAME}")
print(f"   Model Name: {MODEL_NAME}")
print(f"   Artifact Path: {MODEL_ARTIFACT_PATH}")
print(f"   Metric Key: {METRIC_KEY}")
print(f"   Improvement Threshold: {IMPROVEMENT_THRESHOLD * 100}%")
print("=" * 80)

# =============================================================================
# ✅ INITIALIZATION
# =============================================================================
try:
    spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    print("✅ MLflow and Spark initialized\n")

    # Verify experiment exists
    exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if exp is None:
        print(f"❌ ERROR: Experiment '{EXPERIMENT_NAME}' not found!")
        print("\n💡 Available experiments:")
        all_exps = client.search_experiments(max_results=20)
        for e in all_exps:
            print(f"   - {e.name}")
        print(f"\n⚠️ Please ensure the experiment name matches your training script!")
        sys.exit(1)
    
    print(f"✅ Experiment found: {EXPERIMENT_NAME}")
    print(f"   Experiment ID: {exp.experiment_id}\n")
    mlflow.set_experiment(EXPERIMENT_NAME)

except Exception as e:
    print(f"❌ Initialization failed: {e}")
    traceback.print_exc()
    sys.exit(1)


# =============================================================================
# 📊 STEP 1: GET BEST MODEL FROM ALL EXPERIMENT RUNS
# =============================================================================
def get_best_model_from_experiment():
    """Find the best performing model from all experiment runs"""
    print(f"\n{'='*70}")
    print("📋 STEP 1: Finding BEST Model Across ALL Experiment Runs")
    print(f"{'='*70}")

    try:
        exp = client.get_experiment_by_name(EXPERIMENT_NAME)
        print(f"✅ Experiment: {EXPERIMENT_NAME}")
        print(f"   Experiment ID: {exp.experiment_id}")

        # Get ALL runs with valid metrics, sorted by RMSE (best first)
        all_runs = client.search_runs(
            [exp.experiment_id],
            filter_string=f"metrics.{METRIC_KEY} > 0",
            order_by=[f"metrics.{METRIC_KEY} ASC"],
            max_results=1000
        )

        if not all_runs:
            print(f"\n❌ ERROR: No runs found with valid '{METRIC_KEY}' metric!")
            print("\n💡 Troubleshooting:")
            print("   1. Verify training script completed successfully")
            print("   2. Check that models were logged with test_rmse metric")
            print("   3. Run training_script.py first to generate runs")
            return None

        print(f"✅ Total runs in experiment: {len(all_runs)}")

        best_run = all_runs[0]

        # Show top 10 models
        print(f"\n📊 Top 10 Models in Experiment (by {METRIC_KEY}):")
        print(f"{'Rank':<6} {'Run Name':<40} {'RMSE':<15} {'Timestamp':<20}")
        print("-" * 100)

        for i, run in enumerate(all_runs[:10], 1):
            run_name = run.info.run_name or "Unnamed"
            metric_val = run.data.metrics.get(METRIC_KEY, float('inf'))
            timestamp = datetime.fromtimestamp(run.info.start_time/1000).strftime('%Y-%m-%d %H:%M')
            marker = "👑 BEST" if i == 1 else f"{i}."
            print(f"{marker:<6} {run_name:<40} {metric_val:<15.6f} {timestamp}")

        run_id = best_run.info.run_id
        run_name = best_run.info.run_name or "Unnamed"
        metrics = best_run.data.metrics
        params = best_run.data.params
        metric_value = metrics.get(METRIC_KEY)

        print(f"\n✅ BEST Model Selected:")
        print(f"   Run ID: {run_id}")
        print(f"   Run Name: {run_name}")
        print(f"   {METRIC_KEY}: {metric_value:.6f}")
        print(f"   Rank: #1 out of {len(all_runs)} total runs")
        print(f"   Timestamp: {datetime.fromtimestamp(best_run.info.start_time/1000)}")
        print(f"   Parameters: {dict(list(params.items())[:5])}...")

        return {
            'run_id': run_id,
            'run_name': run_name,
            'metric': metric_value,
            'params': params,
            'metrics_all': metrics,
            'timestamp': best_run.info.start_time,
            'total_runs': len(all_runs)
        }

    except Exception as e:
        print(f"❌ Error getting best model: {e}")
        traceback.print_exc()
        return None


# =============================================================================
# 🏆 STEP 2: GET CURRENT BEST MODEL FROM REGISTRY
# =============================================================================
def get_current_best_model():
    """Get the current best model from model registry"""
    print(f"\n{'='*70}")
    print("📋 STEP 2: Checking Current Best Model in Registry")
    print(f"{'='*70}")

    try:
        # Search for models with Staging or Production alias
        versions = client.search_model_versions(f"name = '{MODEL_NAME}'")
        
        if not versions:
            print("ℹ️ No models found in registry (this is the first model)")
            return None

        # Find best version (prefer Production, then Staging)
        best_version = None
        for v in versions:
            if v.aliases and PRODUCTION_ALIAS in v.aliases:
                best_version = v
                print(f"✅ Found Production model: Version {v.version}")
                break
        
        if not best_version:
            for v in versions:
                if v.aliases and STAGING_ALIAS in v.aliases:
                    best_version = v
                    print(f"✅ Found Staging model: Version {v.version}")
                    break
        
        if not best_version and versions:
            best_version = versions[0]
            print(f"✅ Found latest model: Version {best_version.version}")

        if best_version:
            run = client.get_run(best_version.run_id)
            metric = run.data.metrics.get(METRIC_KEY)
            
            print(f"   Version: {best_version.version}")
            print(f"   Run ID: {best_version.run_id}")
            print(f"   {METRIC_KEY}: {metric:.6f}")
            print(f"   Aliases: {', '.join(best_version.aliases) if best_version.aliases else 'None'}")
            
            return {
                'version': best_version.version,
                'run_id': best_version.run_id,
                'metric': metric,
                'aliases': best_version.aliases or []
            }
        
        return None

    except Exception as e:
        print(f"ℹ️ No current model in registry: {e}")
        return None


# =============================================================================
# 🔍 STEP 3: COMPARE MODELS
# =============================================================================
def compare_models(new_model, current_model):
    """Compare new model with current best model"""
    print(f"\n{'='*70}")
    print("📋 STEP 3: Model Comparison")
    print(f"{'='*70}")

    if not current_model:
        print("✅ No existing model - promoting first model to Staging")
        return True, "First model registration", 0.0

    new_metric = new_model['metric']
    current_metric = current_model['metric']
    
    improvement = (current_metric - new_metric) / current_metric
    improvement_pct = improvement * 100

    print(f"\n📊 Comparison Results:")
    print(f"   New Model RMSE: {new_metric:.6f}")
    print(f"   Current Model RMSE: {current_metric:.6f}")
    print(f"   Improvement: {improvement_pct:.2f}%")
    print(f"   Required Threshold: {IMPROVEMENT_THRESHOLD * 100}%")

    if improvement >= IMPROVEMENT_THRESHOLD:
        print(f"\n✅ PROMOTE: New model is {improvement_pct:.2f}% better")
        return True, f"Improved by {improvement_pct:.2f}%", improvement
    else:
        print(f"\n❌ DO NOT PROMOTE: Improvement ({improvement_pct:.2f}%) below threshold")
        return False, f"Insufficient improvement ({improvement_pct:.2f}%)", improvement


# =============================================================================
# 🚀 STEP 4: PROMOTE TO STAGING
# =============================================================================
def promote_to_staging(best_model, comparison_result):
    """Promote the best model to Staging"""
    print(f"\n{'='*70}")
    print("📋 STEP 4: Promoting Model to Staging")
    print(f"{'='*70}")

    try:
        model_uri = f"runs:/{best_model['run_id']}/{MODEL_ARTIFACT_PATH}"
        
        # Register the model
        print(f"⏳ Registering model from run: {best_model['run_id']}")
        new_version = mlflow.register_model(model_uri, MODEL_NAME)
        
        print(f"✅ Model registered as Version {new_version.version}")
        
        # Set Staging alias
        print(f"⏳ Setting '{STAGING_ALIAS}' alias...")
        client.set_registered_model_alias(
            MODEL_NAME, 
            STAGING_ALIAS, 
            new_version.version
        )
        
        print(f"✅ Version {new_version.version} promoted to @{STAGING_ALIAS}")
        
        # Add tags
        tags = {
            "promoted_from": "evaluation_pipeline",
            "improvement_pct": f"{comparison_result['improvement'] * 100:.2f}",
            "evaluation_timestamp": datetime.now().isoformat(),
            "metric_rmse": str(best_model['metric'])
        }
        
        for key, value in tags.items():
            client.set_model_version_tag(MODEL_NAME, new_version.version, key, value)
        
        print(f"✅ Tags added to Version {new_version.version}")
        
        return new_version.version

    except Exception as e:
        print(f"❌ Promotion failed: {e}")
        traceback.print_exc()
        return None


# =============================================================================
# 📝 STEP 5: LOG COMPARISON TO DELTA TABLE
# =============================================================================
def log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version):
    """Log evaluation results to Delta table"""
    print(f"\n{'='*70}")
    print("📋 STEP 5: Logging Evaluation Results")
    print(f"{'='*70}")

    try:
        log_data = {
            "evaluation_timestamp": [datetime.now()],
            "new_run_id": [new_model['run_id']],
            "new_run_name": [new_model['run_name']],
            "new_metric": [new_model['metric']],
            "current_version": [current_model['version'] if current_model else None],
            "current_metric": [current_model['metric'] if current_model else None],
            "improvement_pct": [comparison_result['improvement'] * 100],
            "promoted": [comparison_result['should_promote']],
            "promoted_version": [promoted_version],
            "reason": [comparison_result['reason']]
        }
        
        df = spark.createDataFrame(pd.DataFrame(log_data))
        
        # Append to Delta table
        df.write.format("delta").mode("append").saveAsTable(COMPARISON_LOG_TABLE)
        
        print(f"✅ Evaluation logged to {COMPARISON_LOG_TABLE}")

    except Exception as e:
        print(f"⚠️ Failed to log to Delta: {e}")


# =============================================================================
# 🎬 MAIN EXECUTION
# =============================================================================
def main():
    print(f"\n🎯 Evaluation Strategy: ALL-TIME BEST")
    print(f"   Experiment: {EXPERIMENT_NAME}")
    print(f"   Model Registry: {MODEL_NAME}")
    print(f"   Metric: {METRIC_KEY} (lower is better)")
    
    # Step 1: Find best model from experiment
    best_model = get_best_model_from_experiment()
    if not best_model:
        print("\n❌ EVALUATION FAILED - No valid runs found")
        sys.exit(1)

    # Step 2: Get current best model
    current_model = get_current_best_model()
    
    # Step 3: Compare models
    should_promote, reason, improvement = compare_models(best_model, current_model)
    comparison_result = {
        'should_promote': should_promote,
        'reason': reason,
        'improvement': improvement
    }

    # Step 4: Promote if needed
    promoted_version = None
    if should_promote:
        promoted_version = promote_to_staging(best_model, comparison_result)
    
    # Step 5: Log results
    log_comparison_to_delta(best_model, current_model, comparison_result, promoted_version)

    # Final summary
    print("\n" + "=" * 80)
    print("✅ MODEL EVALUATION COMPLETE")
    print("=" * 80)
    print(f"Decision: {'PROMOTED ✅' if should_promote else 'NOT PROMOTED ❌'}")
    print(f"Reason: {reason}")
    print(f"Selected: {best_model['run_name']} (Rank #1 from {best_model['total_runs']} runs)")
    print(f"RMSE: {best_model['metric']:.6f}")
    if promoted_version:
        print(f"Promoted Version: v{promoted_version} → @{STAGING_ALIAS}")
    print("=" * 80)
    
    sys.exit(0 if should_promote else 1)


if __name__ == "__main__":
    main()
    
    
    
    

# # Databricks notebook source
# # =============================================================================
# # 🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM
# # =============================================================================
# # This script compares newly trained model with current best model
# # Auto-promotes if better, sends notifications, logs everything
# # =============================================================================

# %pip install xgboost requests
# import mlflow
# from mlflow.tracking import MlflowClient
# import pandas as pd
# import numpy as np
# import json
# import sys
# import os
# from datetime import datetime
# from pyspark.sql import SparkSession
# import requests
# import traceback

# print("=" * 80)
# print("🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM")
# print("=" * 80)

# # =============================================================================
# # ✅ CONFIGURATION (ALIGNED WITH TRAINING SCRIPT)
# # =============================================================================
# EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"
# MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

# STAGING_ALIAS = "staging"   # 🔄 aligned lowercase alias for consistency
# PRODUCTION_ALIAS = "production"

# MODEL_ARTIFACT_PATH = "xgboost_model"   # ✅ exactly same as training script

# METRIC_KEY = "test_rmse"
# IMPROVEMENT_THRESHOLD = 0.02  # 2% improvement needed for promotion

# # Logging Config
# COMPARISON_LOG_TABLE = "workspace.default.model_evaluation_log"

# # =============================================================================
# # ✅ INITIALIZATION
# # =============================================================================
# try:
#     spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
#     mlflow.set_tracking_uri("databricks")
#     mlflow.set_registry_uri("databricks-uc")
#     client = MlflowClient()
#     print("✅ MLflow and Spark initialized\n")
# except Exception as e:
#     print(f"❌ Initialization failed: {e}")
#     sys.exit(1)


# # =============================================================================
# # 📊 STEP 1: GET LATEST TRAINED MODEL FROM EXPERIMENT
# # =============================================================================
# def get_latest_trained_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 1: Finding Latest Trained Model (Metric-driven)")
#     print(f"{'='*70}")

#     try:
#         exp = client.get_experiment_by_name(EXPERIMENT_NAME)
#         if not exp:
#             raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found")

#         runs = client.search_runs(
#             [exp.experiment_id],
#             order_by=["metrics." + METRIC_KEY + " DESC"],  # Fetch best metric, not latest timestamp
#             max_results=1
#         )

#         if not runs:
#             raise ValueError("No runs found in experiment")

#         best_run = runs[0]
#         run_id = best_run.info.run_id
#         run_name = best_run.info.run_name or "Unnamed"
#         metrics = best_run.data.metrics
#         params = best_run.data.params
#         metric_value = metrics.get(METRIC_KEY)

#         print(f"\n✅ Best Training Run Found (by {METRIC_KEY}):")
#         print(f"   Run ID: {run_id}")
#         print(f"   Run Name: {run_name}")
#         print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#         print(f"   Parameters: {dict(list(params.items())[:3])}...")
#         print(f"   Timestamp: {datetime.fromtimestamp(best_run.info.start_time/1000)}")

#         return {
#             'run_id': run_id,
#             'run_name': run_name,
#             'metric': metric_value,
#             'params': params,
#             'metrics_all': metrics,
#             'timestamp': best_run.info.start_time
#         }

#     except Exception as e:
#         print(f"❌ Error getting best model: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 🏆 STEP 2: GET CURRENT BEST MODEL (STAGING/PRODUCTION)
# # =============================================================================
# def get_current_best_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 2: Finding Current Best Model in Registry")
#     print(f"{'='*70}")

#     best_model = None
#     for alias_name in [PRODUCTION_ALIAS, STAGING_ALIAS]:
#         try:
#             mv = client.get_model_version_by_alias(MODEL_NAME, alias_name)
#             run = client.get_run(mv.run_id)
#             metric_value = run.data.metrics.get(METRIC_KEY)
#             if metric_value is None:
#                 metric_tag = mv.tags.get("metric_rmse")
#                 metric_value = float(metric_tag) if metric_tag else None

#             best_model = {
#                 'version': mv.version,
#                 'run_id': mv.run_id,
#                 'alias': alias_name,
#                 'metric': metric_value,
#                 'params': run.data.params,
#                 'metrics_all': run.data.metrics
#             }

#             print(f"\n✅ Found Model with @{alias_name} Alias:")
#             print(f"   Version: v{mv.version}")
#             print(f"   Run ID: {mv.run_id}")
#             print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#             break
#         except Exception:
#             print(f"   No model found with @{alias_name} alias")
#             continue

#     if not best_model:
#         print("\nℹ️ No existing model in registry. This will be the first model.")

#     return best_model


# # =============================================================================
# # ⚖️ STEP 3: COMPARE MODELS
# # =============================================================================
# def compare_models(new_model, current_model):
#     print(f"\n{'='*70}")
#     print("📋 STEP 3: Model Comparison Analysis")
#     print(f"{'='*70}")

#     if current_model is None:
#         print("\n🟢 DECISION: PROMOTE — First model, no existing baseline.")
#         return True, "First model - no comparison needed", None

#     if new_model['metric'] is None:
#         print("\n🔴 DECISION: DO NOT PROMOTE — Missing new model metric.")
#         return False, "New model missing metric", None

#     if current_model['metric'] is None:
#         print("\n🟢 DECISION: PROMOTE — Current model lacks metric.")
#         return True, "Current model lacks metric", None

#     new_metric = new_model['metric']
#     current_metric = current_model['metric']

#     improvement = current_metric - new_metric
#     improvement_pct = (improvement / current_metric) * 100

#     print(f"\n📊 Comparison Summary:")
#     print(f"   New RMSE: {new_metric:.6f}")
#     print(f"   Old RMSE: {current_metric:.6f}")
#     print(f"   Improvement: {improvement:.6f} ({improvement_pct:+.2f}%)")

#     threshold_value = current_metric * IMPROVEMENT_THRESHOLD

#     if improvement > threshold_value:
#         print(f"\n🟢 PROMOTE — New model {improvement_pct:.2f}% better.")
#         return True, f"Improved by {improvement_pct:.2f}%", improvement_pct
#     elif abs(improvement) <= threshold_value:
#         print(f"\n🟡 NO PROMOTION — Similar performance.")
#         return False, f"Similar performance ({improvement_pct:+.2f}%)", improvement_pct
#     else:
#         print(f"\n🔴 DO NOT PROMOTE — Worse performance.")
#         return False, f"Worse by {abs(improvement_pct):.2f}%", improvement_pct


# # =============================================================================
# # 🚀 STEP 4: PROMOTE TO STAGING
# # =============================================================================
# def promote_to_staging(new_model, comparison_result):
#     print(f"\n{'='*70}")
#     print("📋 STEP 4: Register & Promote to Staging")
#     print(f"{'='*70}")

#     try:
#         model_uri = f"runs:/{new_model['run_id']}/{MODEL_ARTIFACT_PATH}"
#         print(f"Registering model from URI → {model_uri}")

#         new_version = mlflow.register_model(model_uri, MODEL_NAME)

#         client.set_model_version_tag(MODEL_NAME, new_version.version, "source_run_id", new_model['run_id'])
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "metric_rmse", str(new_model['metric']))
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "promotion_reason", comparison_result['reason'])

#         client.set_registered_model_alias(MODEL_NAME, STAGING_ALIAS, new_version.version)

#         print(f"\n✅ Model Registered & Promoted → @{STAGING_ALIAS}")
#         print(f"   Version: v{new_version.version}")
#         print(f"   RMSE: {new_model['metric']:.6f}")
#         print(f"   Reason: {comparison_result['reason']}")
#         return new_version.version

#     except Exception as e:
#         print(f"\n❌ Promotion failed: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 📝 STEP 5: LOG RESULTS
# # =============================================================================
# def log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version=None):
#     try:
#         log_data = {
#             'timestamp': datetime.now(),
#             'new_run_id': new_model['run_id'],
#             'new_run_name': new_model['run_name'],
#             'new_metric': new_model['metric'],
#             'current_version': int(current_model['version']) if current_model else None,
#             'current_metric': current_model['metric'] if current_model else None,
#             'current_alias': current_model['alias'] if current_model else None,
#             'should_promote': comparison_result['should_promote'],
#             'promotion_reason': comparison_result['reason'],
#             'improvement_pct': comparison_result['improvement'],
#             'promoted_to_staging': promoted_version is not None,
#             'promoted_version': int(promoted_version) if promoted_version else None,
#             'threshold_used': IMPROVEMENT_THRESHOLD * 100
#         }

#         spark.createDataFrame(pd.DataFrame([log_data])) \
#             .write.format("delta").mode("append").option("mergeSchema", "true") \
#             .saveAsTable(COMPARISON_LOG_TABLE)

#         print(f"✅ Logged to {COMPARISON_LOG_TABLE}")

#     except Exception as e:
#         print(f"⚠️ Logging failed: {e}")


# # =============================================================================
# # 🎬 MAIN EXECUTION
# # =============================================================================
# def main():
#     new_model = get_latest_trained_model()
#     if not new_model:
#         print("❌ No new model found.")
#         sys.exit(1)

#     current_model = get_current_best_model()
#     should_promote, reason, improvement = compare_models(new_model, current_model)
#     comparison_result = {
#         'should_promote': should_promote,
#         'reason': reason,
#         'improvement': improvement
#     }

#     promoted_version = promote_to_staging(new_model, comparison_result) if should_promote else None
#     log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version)

#     print("\n" + "=" * 80)
#     print("✅ MODEL EVALUATION COMPLETE")
#     print("=" * 80)
#     print(f"Decision: {'PROMOTED' if should_promote else 'NOT PROMOTED'}")
#     print(f"Reason: {reason}")
#     if promoted_version:
#         print(f"Promoted Version: v{promoted_version} → @{STAGING_ALIAS}")
#     print("=" * 80)


# # =============================================================================
# # ✅ EXECUTE
# # =============================================================================
# if __name__ == "__main__":
#     main()





# # Databricks notebook source
# # =============================================================================
# # 🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM
# # =============================================================================
# # This script compares newly trained model with current best model
# # Auto-promotes if better, sends notifications, logs everything
# # =============================================================================

# %pip install xgboost requests
# import mlflow
# from mlflow.tracking import MlflowClient
# import pandas as pd
# import numpy as np
# import json
# import sys
# import os
# from datetime import datetime
# from pyspark.sql import SparkSession
# import requests
# import traceback

# print("=" * 80)
# print("🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM")
# print("=" * 80)

# # =============================================================================
# # ✅ CONFIGURATION (ALIGNED WITH TRAINING SCRIPT)
# # =============================================================================
# EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"
# MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

# STAGING_ALIAS = "staging"   # 🔄 aligned lowercase alias for consistency
# PRODUCTION_ALIAS = "production"

# MODEL_ARTIFACT_PATH = "xgboost_model"   # ✅ exactly same as training script

# METRIC_KEY = "test_rmse"
# IMPROVEMENT_THRESHOLD = 0.02  # 2% improvement needed for promotion

# # Notification & Logging Config
# ENABLE_SLACK = False
# SLACK_WEBHOOK_URL = ""
# ENABLE_EMAIL = False
# EMAIL_RECIPIENT = ""
# COMPARISON_LOG_TABLE = "workspace.default.model_evaluation_log"

# # =============================================================================
# # ✅ INITIALIZATION
# # =============================================================================
# try:
#     spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
#     mlflow.set_tracking_uri("databricks")
#     mlflow.set_registry_uri("databricks-uc")
#     client = MlflowClient()
#     print("✅ MLflow and Spark initialized\n")
# except Exception as e:
#     print(f"❌ Initialization failed: {e}")
#     sys.exit(1)


# # =============================================================================
# # 📊 STEP 1: GET LATEST TRAINED MODEL FROM EXPERIMENT
# # =============================================================================
# def get_latest_trained_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 1: Finding Latest Trained Model")
#     print(f"{'='*70}")

#     try:
#         exp = client.get_experiment_by_name(EXPERIMENT_NAME)
#         if not exp:
#             raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found")

#         runs = client.search_runs(
#             [exp.experiment_id],
#             order_by=["start_time DESC"],
#             max_results=1
#         )

#         if not runs:
#             raise ValueError("No runs found in experiment")

#         latest_run = runs[0]
#         run_id = latest_run.info.run_id
#         run_name = latest_run.info.run_name or "Unnamed"
#         metrics = latest_run.data.metrics
#         params = latest_run.data.params
#         metric_value = metrics.get(METRIC_KEY)

#         print(f"\n✅ Latest Training Run Found:")
#         print(f"   Run ID: {run_id}")
#         print(f"   Run Name: {run_name}")
#         print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#         print(f"   Parameters: {dict(list(params.items())[:3])}...")
#         print(f"   Timestamp: {datetime.fromtimestamp(latest_run.info.start_time/1000)}")

#         return {
#             'run_id': run_id,
#             'run_name': run_name,
#             'metric': metric_value,
#             'params': params,
#             'metrics_all': metrics,
#             'timestamp': latest_run.info.start_time
#         }

#     except Exception as e:
#         print(f"❌ Error getting latest model: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 🏆 STEP 2: GET CURRENT BEST MODEL (STAGING/PRODUCTION)
# # =============================================================================
# def get_current_best_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 2: Finding Current Best Model in Registry")
#     print(f"{'='*70}")

#     best_model = None
#     for alias_name in [PRODUCTION_ALIAS, STAGING_ALIAS]:
#         try:
#             mv = client.get_model_version_by_alias(MODEL_NAME, alias_name)
#             run = client.get_run(mv.run_id)
#             metric_value = run.data.metrics.get(METRIC_KEY)
#             if metric_value is None:
#                 metric_tag = mv.tags.get("metric_rmse")
#                 metric_value = float(metric_tag) if metric_tag else None

#             best_model = {
#                 'version': mv.version,
#                 'run_id': mv.run_id,
#                 'alias': alias_name,
#                 'metric': metric_value,
#                 'params': run.data.params,
#                 'metrics_all': run.data.metrics
#             }

#             print(f"\n✅ Found Model with @{alias_name} Alias:")
#             print(f"   Version: v{mv.version}")
#             print(f"   Run ID: {mv.run_id}")
#             print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#             break
#         except Exception:
#             print(f"   No model found with @{alias_name} alias")
#             continue

#     if not best_model:
#         print("\nℹ️ No existing model in registry. This will be the first model.")

#     return best_model


# # =============================================================================
# # ⚖️ STEP 3: COMPARE MODELS
# # =============================================================================
# def compare_models(new_model, current_model):
#     print(f"\n{'='*70}")
#     print("📋 STEP 3: Model Comparison Analysis")
#     print(f"{'='*70}")

#     if current_model is None:
#         print("\n🟢 DECISION: PROMOTE — First model, no existing baseline.")
#         return True, "First model - no comparison needed", None

#     if new_model['metric'] is None:
#         print("\n🔴 DECISION: DO NOT PROMOTE — Missing new model metric.")
#         return False, "New model missing metric", None

#     if current_model['metric'] is None:
#         print("\n🟢 DECISION: PROMOTE — Current model lacks metric.")
#         return True, "Current model lacks metric", None

#     new_metric = new_model['metric']
#     current_metric = current_model['metric']

#     improvement = current_metric - new_metric
#     improvement_pct = (improvement / current_metric) * 100

#     print(f"\n📊 Comparison Summary:")
#     print(f"   New RMSE: {new_metric:.6f}")
#     print(f"   Old RMSE: {current_metric:.6f}")
#     print(f"   Improvement: {improvement:.6f} ({improvement_pct:+.2f}%)")

#     threshold_value = current_metric * IMPROVEMENT_THRESHOLD

#     if improvement > threshold_value:
#         print(f"\n🟢 PROMOTE — New model {improvement_pct:.2f}% better.")
#         return True, f"Improved by {improvement_pct:.2f}%", improvement_pct
#     elif abs(improvement) <= threshold_value:
#         print(f"\n🟡 NO PROMOTION — Similar performance.")
#         return False, f"Similar performance ({improvement_pct:+.2f}%)", improvement_pct
#     else:
#         print(f"\n🔴 DO NOT PROMOTE — Worse performance.")
#         return False, f"Worse by {abs(improvement_pct):.2f}%", improvement_pct


# # =============================================================================
# # 🚀 STEP 4: PROMOTE TO STAGING
# # =============================================================================
# def promote_to_staging(new_model, comparison_result):
#     print(f"\n{'='*70}")
#     print("📋 STEP 4: Register & Promote to Staging")
#     print(f"{'='*70}")

#     try:
#         model_uri = f"runs:/{new_model['run_id']}/{MODEL_ARTIFACT_PATH}"
#         print(f"Registering model from URI → {model_uri}")

#         new_version = mlflow.register_model(model_uri, MODEL_NAME)

#         client.set_model_version_tag(MODEL_NAME, new_version.version, "source_run_id", new_model['run_id'])
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "metric_rmse", str(new_model['metric']))
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "promotion_reason", comparison_result['reason'])

#         client.set_registered_model_alias(MODEL_NAME, STAGING_ALIAS, new_version.version)

#         print(f"\n✅ Model Registered & Promoted → @{STAGING_ALIAS}")
#         print(f"   Version: v{new_version.version}")
#         print(f"   RMSE: {new_model['metric']:.6f}")
#         print(f"   Reason: {comparison_result['reason']}")
#         return new_version.version

#     except Exception as e:
#         print(f"\n❌ Promotion failed: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 📝 STEP 5: LOG RESULTS
# # =============================================================================
# def log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version=None):
#     try:
#         log_data = {
#             'timestamp': datetime.now(),
#             'new_run_id': new_model['run_id'],
#             'new_run_name': new_model['run_name'],
#             'new_metric': new_model['metric'],
#             'current_version': int(current_model['version']) if current_model else None,
#             'current_metric': current_model['metric'] if current_model else None,
#             'current_alias': current_model['alias'] if current_model else None,
#             'should_promote': comparison_result['should_promote'],
#             'promotion_reason': comparison_result['reason'],
#             'improvement_pct': comparison_result['improvement'],
#             'promoted_to_staging': promoted_version is not None,
#             'promoted_version': int(promoted_version) if promoted_version else None,
#             'threshold_used': IMPROVEMENT_THRESHOLD * 100
#         }

#         spark.createDataFrame(pd.DataFrame([log_data])) \
#             .write.format("delta").mode("append").option("mergeSchema", "true") \
#             .saveAsTable(COMPARISON_LOG_TABLE)

#         print(f"✅ Logged to {COMPARISON_LOG_TABLE}")

#     except Exception as e:
#         print(f"⚠️ Logging failed: {e}")


# # =============================================================================
# # 🎬 MAIN EXECUTION
# # =============================================================================
# def main():
#     new_model = get_latest_trained_model()
#     if not new_model:
#         print("❌ No new model found.")
#         sys.exit(1)

#     current_model = get_current_best_model()
#     should_promote, reason, improvement = compare_models(new_model, current_model)
#     comparison_result = {
#         'should_promote': should_promote,
#         'reason': reason,
#         'improvement': improvement
#     }

#     promoted_version = promote_to_staging(new_model, comparison_result) if should_promote else None
#     log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version)

#     print("\n" + "=" * 80)
#     print("✅ MODEL EVALUATION COMPLETE")
#     print("=" * 80)
#     print(f"Decision: {'PROMOTED' if should_promote else 'NOT PROMOTED'}")
#     print(f"Reason: {reason}")
#     if promoted_version:
#         print(f"Promoted Version: v{promoted_version} → @{STAGING_ALIAS}")
#     print("=" * 80)


# # =============================================================================
# # ✅ EXECUTE
# # =============================================================================
# if __name__ == "__main__":
#     main()
